## Installing Requirements

In [1]:
! pip install torch
! pip install transformers
! pip install sentencepiece
! pip install alive-progress

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210095 sha256=fde6c214df06608f0f35394a5e213851d5ecb432b2a2dbb002c39b331724f56d
  Stored in directory: /root/.cache/pip/wheels/01/e1/49/37e6bde9886439057450c494a79b0bef8bbe897a54aebfc757
Successfully built grapheme


## Loading The Model

In [2]:
import torch
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
import numpy as np

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
model_name = "persiannlp/mt5-small-parsinlu-sentiment-analysis"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

In [4]:
def model_predict(text_a, text_b):
    features = tokenizer( [(text_a, text_b)], padding="max_length", truncation=True, return_tensors='pt')
    output = model(**features)
    logits = output[0]
    probs = torch.nn.functional.softmax(logits, dim=1).tolist()
    idx = np.argmax(np.array(probs))
    print(labels[idx], probs)

In [5]:
def run_model(context, query, **generator_args):
    input_ids = tokenizer.encode(context + "<sep>" + query, return_tensors="pt")
    res = model.generate(input_ids, **generator_args)
    output = tokenizer.batch_decode(res, skip_special_tokens=True)
#     print(output)
    return output

## Creating Dataset

In [6]:
input_file = "/kaggle/input/parsinlu-foods-dataset/food.jsonl"

### Creating Dataframe

In [7]:
with open(input_file) as f:
    lines = f.read().splitlines()

In [8]:
import pandas as pd

df_inter = pd.DataFrame(lines)
df_inter.columns = ['json_element']

In [9]:
import json

df_inter['json_element'].apply(json.loads)

df_final = pd.json_normalize(df_inter['json_element'].apply(json.loads))

df_final.head(n=5)

,review,sentiment,category,aspects.طعم,aspects.کیفیت,aspects.ارزش غذایی,aspects.ارزش خرید,aspects.بسته بندی,aspects.ارسال
0,من یه مدته فقط از این محصول استفاده میکنم اما ...,3,حبوبات و سویا,NaN,NaN,NaN,NaN,NaN,NaN
1,اگه بخوای به صورت مکمل استفاده اش کنی خوبه وار...,1,شربت و آبمیوه,NaN,NaN,NaN,NaN,NaN,NaN
2,طعم خیلی خوبی داره و توضیحات کالا کاملا با خود...,2,تن ماهی,2,NaN,NaN,NaN,NaN,NaN
3,من این رو که گرفتم خیلی به نظر مونده بود و بشد...,-2,گوشت مرغ,-2,-2,NaN,NaN,NaN,NaN
4,کلا که نوشابه محصول مفیدی نیست. اما اونایی که ...,1,نوشابه,NaN,NaN,-1,1,NaN,NaN


In [10]:
df_final['sentiment'].value_counts()

-2    462
2     440
1     420
-1    381
3     184
0      30
Name: sentiment, dtype: int64

In [11]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1917 entries, 0 to 1916
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   review              1917 non-null   object
 1   sentiment           1917 non-null   object
 2   category            1917 non-null   object
 3   aspects.طعم         886 non-null    object
 4   aspects.کیفیت       553 non-null    object
 5   aspects.ارزش غذایی  89 non-null     object
 6   aspects.ارزش خرید   414 non-null    object
 7   aspects.بسته بندی   156 non-null    object
 8   aspects.ارسال       64 non-null     object
dtypes: object(9)
memory usage: 134.9+ KB


In [12]:
df_final.describe()

,review,sentiment,category,aspects.طعم,aspects.کیفیت,aspects.ارزش غذایی,aspects.ارزش خرید,aspects.بسته بندی,aspects.ارسال
count,1917,1917,1917,886,553,89,414,156,64
unique,1917,6,20,6,6,6,6,5,4
top,من یه مدته فقط از این محصول استفاده میکنم اما ...,-2,تن ماهی,1,1,1,1,1,-1
freq,1,462,106,269,179,39,176,62,24


In [13]:
df_final = df_final[['review', 'sentiment']]
df_final.head(n=5)

,review,sentiment
0,من یه مدته فقط از این محصول استفاده میکنم اما ...,3
1,اگه بخوای به صورت مکمل استفاده اش کنی خوبه وار...,1
2,طعم خیلی خوبی داره و توضیحات کالا کاملا با خود...,2
3,من این رو که گرفتم خیلی به نظر مونده بود و بشد...,-2
4,کلا که نوشابه محصول مفیدی نیست. اما اونایی که ...,1


In [14]:
random_state = 18

### Samples

In [15]:
samples_count = 1
sample_prompts = df_final.groupby('sentiment').apply(lambda x: x.sample(n=samples_count, random_state=random_state))
sample_prompts

,,review,sentiment
sentiment,,,
-1,17,پودر قهوه دارای کیفیت مطلوب نبود وعطر و بوی در...,-1
-2,836,از این برند انتظار بیشتری داشتم خیلی خشک و بی ...,-2
0,1015,تعداد شیر های پرچرب کمتر از کم چرب بود ۴ تا کم...,0
1,238,دفعه های قبل که سفارش دادم بو میداد ولی ایندفع...,1
2,499,به نظرم خیلی خوشمزه بود طعم شکلات با قهوه عالی...,2
3,916,از نظر کیفیت خوبه ولی از نظر قیمت فقط تو پیشنه...,3


## Prompts

In [16]:
import time

max_retries = 5

In [17]:
label_dict = {
    "-2" : "NEGATIVE",
    "-1" : "NEGATIVE",
    "0"  : "NEUTRAL",
    "1"  : "POSITIVE",
    "2"  : "POSITIVE",
    "3"  : "OTHER",
}

In [18]:
per_to_eng_dict = {
    "مثبت" : "POSITIVE",
    "خنثی" : "NEUTRAL",
    "منفی" : "NEGATIVE",
    "سایر" : "OTHER",
}

In [19]:
eng_to_per_dict = {
    "POSITIVE"  : "مثبت",
    "NEUTRAL"   : "خنثی",
    "NEGATIVE"  : "منفی",
    "OTHER"     : "سایر",
}

In [20]:
acceptable_classes = [
    "NEGATIVE",
    "NEUTRAL",
    "POSITIVE",
    "OTHER",
]

In [21]:
context_01 = f"""
The below sentence is a person's review. The review is in Persian. Identify the sentiment or polarity associated with it. \
Possible answers are: POSITIVE, NEUTRAL, NEGATIVE, OTHER. \
Use OTHER when the sentence does not include any specific sense, or has mixed or borderline senses. \
"""

In [22]:
query_01 = f"""
Review: SAMPLE_REVIEW
Sentiment:
"""

In [23]:
context_02 = f"""
جمله زیر نظر یک شخص است. این جمله به زبان فارسی است. بار یا احساس موجود در این جمله را شناسایی کن. \
پاسخ‌ های ممکن کلمات روبرو هستند: POSITIVE, NEUTRAL, NEGATIVE, OTHER. \
زمانی که جمله دارای احساس خاصی نیست یا شامل احساسات مختلف است از OTHER استفاده کن \
"""

In [24]:
query_02 = f"""
Review: SAMPLE_REVIEW
Sentiment:
"""

In [25]:
context_03 = f"""
جمله زیر نظر یک شخص است. این جمله به زبان فارسی است. بار یا احساس موجود در این جمله را شناسایی کن. \
پاسخ‌ های ممکن کلمات روبرو هستند: POSITIVE, NEUTRAL, NEGATIVE, OTHER. \
زمانی که جمله دارای احساس خاصی نیست یا شامل احساسات مختلف است از OTHER استفاده کن \
"""

In [26]:
query_03 = f"""
نظر: SAMPLE_REVIEW
احساس:
"""

In [27]:
context_04 = f"""
جمله زیر نظر یک شخص است. این جمله به زبان فارسی است. بار یا احساس موجود در این جمله را شناسایی کن. \
پاسخ‌ های ممکن کلمات روبرو هستند: مثبت، خنثی، منفی، سایر. \
زمانی که جمله دارای احساس خاصی نیست یا شامل احساسات مختلف است از خنثی استفاده کن \
"""

In [28]:
query_04 = f"""
نظر: SAMPLE_REVIEW
احساس:
"""

In [29]:
query_05 = "نظر شما به طور کلی راجع به این خوراکی چیست؟"

## Zero-Shot Sample Test


### Prompt #01

In [30]:
for index, row in sample_prompts.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = run_model(context=context_01, query=query_01.replace("SAMPLE_REVIEW", row['review']))
            print(f"Review: {row['review']}")
            print(f"Model Response: {' '.join(response)}")
            print(f"Actual Label: {label_dict[row['sentiment']]}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(5)

    if retries == max_retries:
        print(f"Failed to process this review after {max_retries} retries. Skipping to the next review.")


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Review: پودر قهوه دارای کیفیت مطلوب نبود وعطر و بوی در فضا پخش نمی شد
Model Response: no sentiment expressed
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: از این برند انتظار بیشتری داشتم خیلی خشک و بی مزه بود
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: تعداد شیر های پرچرب کمتر از کم چرب بود ۴ تا کم چرب بود و ۲ تا پر چرب
Model Response: negative
Actual Label: NEUTRAL
---------------------------------------------------------------------
Review: دفعه های قبل که سفارش دادم بو میداد ولی ایندفعه تازه بود و بوی بد هم نمیداد
Model Response: negative
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: به نظرم خیلی خوشمزه بود طعم شکلات با قهوه عالی میشه
Model Response: negative
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: از نظر کیفیت خوبه ولی از نظر قیمت فق

### Prompt #02

In [31]:
for index, row in sample_prompts.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = run_model(context=context_02, query=query_02.replace("SAMPLE_REVIEW", row['review']))
            print(f"Review: {row['review']}")
            print(f"Model Response: {' '.join(response)}")
            print(f"Actual Label: {label_dict[row['sentiment']]}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(5)

    if retries == max_retries:
        print(f"Failed to process this review after {max_retries} retries. Skipping to the next review.")


Review: پودر قهوه دارای کیفیت مطلوب نبود وعطر و بوی در فضا پخش نمی شد
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: از این برند انتظار بیشتری داشتم خیلی خشک و بی مزه بود
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: تعداد شیر های پرچرب کمتر از کم چرب بود ۴ تا کم چرب بود و ۲ تا پر چرب
Model Response: negative
Actual Label: NEUTRAL
---------------------------------------------------------------------
Review: دفعه های قبل که سفارش دادم بو میداد ولی ایندفعه تازه بود و بوی بد هم نمیداد
Model Response: negative
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: به نظرم خیلی خوشمزه بود طعم شکلات با قهوه عالی میشه
Model Response: negative
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: از نظر کیفیت خوبه ولی از نظر قیمت فقط تو پیشنهاد و

### Prompt #03

In [32]:
for index, row in sample_prompts.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = run_model(context=context_03, query=query_03.replace("SAMPLE_REVIEW", row['review']))
            print(f"Review: {row['review']}")
            print(f"Model Response: {' '.join(response)}")
            print(f"Actual Label: {label_dict[row['sentiment']]}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(5)

    if retries == max_retries:
        print(f"Failed to process this review after {max_retries} retries. Skipping to the next review.")


Review: پودر قهوه دارای کیفیت مطلوب نبود وعطر و بوی در فضا پخش نمی شد
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: از این برند انتظار بیشتری داشتم خیلی خشک و بی مزه بود
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: تعداد شیر های پرچرب کمتر از کم چرب بود ۴ تا کم چرب بود و ۲ تا پر چرب
Model Response: negative
Actual Label: NEUTRAL
---------------------------------------------------------------------
Review: دفعه های قبل که سفارش دادم بو میداد ولی ایندفعه تازه بود و بوی بد هم نمیداد
Model Response: negative
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: به نظرم خیلی خوشمزه بود طعم شکلات با قهوه عالی میشه
Model Response: negative
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: از نظر کیفیت خوبه ولی از نظر قیمت فقط تو پیشنهاد و

### Prompt #04

In [33]:
for index, row in sample_prompts.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = run_model(context=context_04, query=query_04.replace("SAMPLE_REVIEW", row['review']))
            print(f"Review: {row['review']}")
            print(f"Model Response: {' '.join(response)}")
            print(f"Actual Label: {label_dict[row['sentiment']]}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(5)

    if retries == max_retries:
        print(f"Failed to process this review after {max_retries} retries. Skipping to the next review.")


Review: پودر قهوه دارای کیفیت مطلوب نبود وعطر و بوی در فضا پخش نمی شد
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: از این برند انتظار بیشتری داشتم خیلی خشک و بی مزه بود
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: تعداد شیر های پرچرب کمتر از کم چرب بود ۴ تا کم چرب بود و ۲ تا پر چرب
Model Response: negative
Actual Label: NEUTRAL
---------------------------------------------------------------------
Review: دفعه های قبل که سفارش دادم بو میداد ولی ایندفعه تازه بود و بوی بد هم نمیداد
Model Response: negative
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: به نظرم خیلی خوشمزه بود طعم شکلات با قهوه عالی میشه
Model Response: negative
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: از نظر کیفیت خوبه ولی از نظر قیمت فقط تو پیشنهاد و

### Prompt #05

In [34]:
for index, row in sample_prompts.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = run_model(context=row['review'], query=query_05)
            print(f"Review: {row['review']}")
            print(f"Model Response: {' '.join(response)}")
            print(f"Actual Label: {label_dict[row['sentiment']]}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(5)

    if retries == max_retries:
        print(f"Failed to process this review after {max_retries} retries. Skipping to the next review.")


Review: پودر قهوه دارای کیفیت مطلوب نبود وعطر و بوی در فضا پخش نمی شد
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: از این برند انتظار بیشتری داشتم خیلی خشک و بی مزه بود
Model Response: very negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: تعداد شیر های پرچرب کمتر از کم چرب بود ۴ تا کم چرب بود و ۲ تا پر چرب
Model Response: negative
Actual Label: NEUTRAL
---------------------------------------------------------------------
Review: دفعه های قبل که سفارش دادم بو میداد ولی ایندفعه تازه بود و بوی بد هم نمیداد
Model Response: negative
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: به نظرم خیلی خوشمزه بود طعم شکلات با قهوه عالی میشه
Model Response: very positive
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: از نظر کیفیت خوبه ولی از نظر قیمت فقط تو

## Metrics

In [35]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from collections import Counter
import numpy as np

def calculate_metrics(actual, predicted):
    accuracy = accuracy_score(actual, predicted)
    f1 = f1_score(actual, predicted, average='weighted', zero_division=0)
    precision = precision_score(actual, predicted, average='weighted', zero_division=0)
    recall = recall_score(actual, predicted, average='weighted', zero_division=0)

    actual_counts = Counter(actual)
    predicted_counts = Counter(predicted)
    classes = list(set(actual + predicted))
    classes.append('All Classes')
    metrics = ['Accuracy', 'F1 Score', 'Precision', 'Recall']

    probability_table = pd.DataFrame(index=metrics, columns=classes)

    for class_label in classes:
        if class_label == 'All Classes':
            probability_table[class_label]["F1 Score"] = f1
            probability_table[class_label]["Precision"] = precision
            probability_table[class_label]["Recall"] = recall
        else:
            class_precision = precision_score(actual, predicted, labels=[class_label], average=None, zero_division=0)
            class_recall = recall_score(actual, predicted, labels=[class_label], average=None, zero_division=0)
            class_f1 = f1_score(actual, predicted, labels=[class_label], average=None, zero_division=0)
            probability_table[class_label]["F1 Score"] = class_f1
            probability_table[class_label]["Precision"] = class_precision
            probability_table[class_label]["Recall"] = class_recall
    probability_table.fillna(value="-", inplace = True)
    probability_table['All Classes']["Accuracy"] = accuracy
    return probability_table

## Test data

In [36]:
test_count = 500
test_df = df_final.sample(n=test_count)
test_df

,review,sentiment
1175,اصلا تازه نبود و بوی ماندگی میداد درصورتی که ت...,-2
1091,از کیفیت و تازگی و بسته بندی وارزش غذایی خوبی ...,2
315,با سلام، کیفیت خوبی داره، البته کمی رقیق است ک...,1
1202,منکه عاشق پاستام نتونسم بخورمش فاجعس همون ماکا...,-2
1242,طعمش اسپایسی هست . برای کسانی که به تندی و سیر...,1
...,...,...
212,عالیه عالی، من عاشق شکلات تلخ و نعناع هستم و ا...,2
1183,چون خیلی از قهوه خوشم نمیاد و نسبتا کم میخورم ...,1
1100,زیاد از حد خرده ریز بود و کمی بو میداد. حتی در...,-2
1054,طعم همه پسندی ندارد از نظر بسته بندی ظاهر مناس...,3


In [37]:
test_df.to_csv('test_df.csv',index=True)

## Zero-Shot Metrics

### Prompt #01

In [38]:
from alive_progress import alive_bar

zero_01_actual = []
zero_01_predicted = []
zero_01_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = run_model(context=context_01, query=query_01.replace("SAMPLE_REVIEW", row['review']))
                response = " ".join(response)
                zero_01_actual.append(label_dict[row['sentiment']])
                zero_01_responses.append(response)

                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        zero_01_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    if "NO SENTIMENT EXPRESSED" in response:
                        zero_01_predicted.append('NEUTRAL')
                    else:
                        zero_01_predicted.append('OTHER')

                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()

on 0: /opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
        warnings.warn(


|████████████████████████████████████████| 500/500 [100%] in 2:00.7 (4.14/s)    


In [39]:
zero_01_results = calculate_metrics(zero_01_actual, zero_01_predicted)
zero_01_results 

,OTHER,NEGATIVE,POSITIVE,NEUTRAL,All Classes
Accuracy,-,-,-,-,0.382
F1 Score,[0.0],[0.6181102362204726],[0.22813688212927757],[0.046242774566473986],0.360292
Precision,[0.0],[0.5181518151815182],[1.0],[0.023952095808383235],0.67873
Recall,[0.0],[0.7658536585365854],[0.12875536480686695],[0.6666666666666666],0.382


In [40]:
np.save('zero_01_actual', zero_01_actual)

In [41]:
np.save('zero_01_predicted', zero_01_predicted)

In [42]:
np.save('zero_01_responses', zero_01_responses)

In [43]:
zero_01_results.to_csv('zero_01_results.csv',index=True)

### Prompt #02

In [44]:
zero_02_actual = []
zero_02_predicted = []
zero_02_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = run_model(context=context_01, query=query_01.replace("SAMPLE_REVIEW", row['review']))
                response = " ".join(response)
                zero_02_actual.append(label_dict[row['sentiment']])
                zero_02_responses.append(response)
                
                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        zero_02_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    if "NO SENTIMENT EXPRESSED" in response:
                        zero_02_predicted.append('NEUTRAL')
                    else:
                        zero_02_predicted.append('OTHER')

                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()

on 0: /opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
        warnings.warn(


|████████████████████████████████████████| 500/500 [100%] in 2:00.5 (4.15/s)    


In [45]:
zero_02_results = calculate_metrics(zero_02_actual, zero_02_predicted)
zero_02_results 

,OTHER,NEGATIVE,POSITIVE,NEUTRAL,All Classes
Accuracy,-,-,-,-,0.382
F1 Score,[0.0],[0.6181102362204726],[0.22813688212927757],[0.046242774566473986],0.360292
Precision,[0.0],[0.5181518151815182],[1.0],[0.023952095808383235],0.67873
Recall,[0.0],[0.7658536585365854],[0.12875536480686695],[0.6666666666666666],0.382


In [46]:
np.save('zero_02_actual', zero_02_actual)

In [47]:
np.save('zero_02_predicted', zero_02_predicted)

In [48]:
np.save('zero_02_responses', zero_02_responses)

In [49]:
zero_02_results.to_csv('zero_02_results.csv',index=True)

### Prompt #03

In [50]:
zero_03_actual = []
zero_03_predicted = []
zero_03_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = run_model(context=context_01, query=query_01.replace("SAMPLE_REVIEW", row['review']))
                response = " ".join(response)
                zero_03_actual.append(label_dict[row['sentiment']])
                zero_03_responses.append(response)
                
                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        zero_03_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    if "NO SENTIMENT EXPRESSED" in response:
                        zero_03_predicted.append('NEUTRAL')
                    else:
                        zero_03_predicted.append('OTHER')

                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()

on 0: /opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
        warnings.warn(


|████████████████████████████████████████| 500/500 [100%] in 2:00.3 (4.15/s)    


In [51]:
zero_03_results = calculate_metrics(zero_03_actual, zero_03_predicted)
zero_03_results

,OTHER,NEGATIVE,POSITIVE,NEUTRAL,All Classes
Accuracy,-,-,-,-,0.382
F1 Score,[0.0],[0.6181102362204726],[0.22813688212927757],[0.046242774566473986],0.360292
Precision,[0.0],[0.5181518151815182],[1.0],[0.023952095808383235],0.67873
Recall,[0.0],[0.7658536585365854],[0.12875536480686695],[0.6666666666666666],0.382


In [52]:
np.save('zero_03_actual', zero_03_actual)

In [53]:
np.save('zero_03_predicted', zero_03_predicted)

In [54]:
np.save('zero_03_responses', zero_03_responses)

In [55]:
zero_03_results.to_csv('zero_03_results.csv',index=True)

### Prompt #04

In [56]:
zero_04_actual = []
zero_04_predicted = []
zero_04_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = run_model(context=context_04, query=query_04.replace("SAMPLE_REVIEW", row['review']))
                response = " ".join(response)
                zero_04_actual.append(label_dict[row['sentiment']])
                zero_04_responses.append(response)
                
                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        zero_04_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    if "NO SENTIMENT EXPRESSED" in response:
                        zero_04_predicted.append('NEUTRAL')
                    else:
                        zero_04_predicted.append('OTHER')

                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()

on 0: /opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
        warnings.warn(


|████████████████████████████████████████| 500/500 [100%] in 1:58.5 (4.22/s)    


In [57]:
zero_04_results = calculate_metrics(zero_04_actual, zero_04_predicted)
zero_04_results

,OTHER,NEGATIVE,POSITIVE,NEUTRAL,All Classes
Accuracy,-,-,-,-,0.558
F1 Score,[0.0],[0.7105263157894736],[0.6345609065155807],[0.07407407407407408],0.58791
Precision,[0.0],[0.6454183266932271],[0.9333333333333333],[0.03875968992248062],0.70002
Recall,[0.0],[0.7902439024390244],[0.48068669527896996],[0.8333333333333334],0.558


In [58]:
np.save('zero_04_actual', zero_04_actual)

In [59]:
np.save('zero_04_predicted', zero_04_predicted)

In [60]:
np.save('zero_04_responses', zero_04_responses)

In [61]:
zero_04_results.to_csv('zero_04_results.csv',index=True)

### Prompt #05

In [62]:
zero_05_actual = []
zero_05_predicted = []
zero_05_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = run_model(context=row['review'], query=query_05)
                response = " ".join(response)
                zero_05_actual.append(label_dict[row['sentiment']])
                zero_05_responses.append(response)
                
                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        zero_05_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    if "NO SENTIMENT EXPRESSED" in response:
                        zero_05_predicted.append('NEUTRAL')
                    else:
                        zero_05_predicted.append('OTHER')

                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()

on 0: /opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
        warnings.warn(


|████████████████████████████████████████| 500/500 [100%] in 1:37.8 (5.11/s)    


In [63]:
zero_05_results = calculate_metrics(zero_05_actual, zero_05_predicted)
zero_05_results

,OTHER,NEGATIVE,POSITIVE,NEUTRAL,All Classes
Accuracy,-,-,-,-,0.908
F1 Score,[0.7200000000000001],[0.9370629370629371],[0.9333333333333333],[0.0],0.899769
Precision,[0.8181818181818182],[0.8973214285714286],[0.9353448275862069],[0.0],0.895409
Recall,[0.6428571428571429],[0.9804878048780488],[0.9313304721030042],[0.0],0.908


In [64]:
np.save('zero_05_actual', zero_05_actual)

In [65]:
np.save('zero_05_predicted', zero_05_predicted)

In [66]:
np.save('zero_05_responses', zero_05_responses)

In [67]:
zero_05_results.to_csv('zero_05_results.csv',index=True)

## Few-Shot Sample Test

In [68]:
def create_examples_df(sample_size=1):
    example_prompts = df_final.groupby('sentiment').apply(lambda x: x.sample(sample_size))
    example_prompts = example_prompts.sample(frac=1).reset_index(drop=True)
    return example_prompts

In [69]:
def create_examples_prompt():
    examples_df = create_examples_df()
    result = ""
    result_persian = ""
    i = 1
    for index, row in examples_df.iterrows():
        result += f"Example {i}:\n"
        result += f"Review: {row['review']}\n"
        result += f"Sentiment: {label_dict[row['sentiment']]}\n"

        result_persian += f"مثال {i}:\n"
        result_persian += f"نظر: {row['review']}\n"
        result_persian += f"احساس: {eng_to_per_dict[label_dict[row['sentiment']]]}\n"

        i += 1

    return result, result_persian

In [70]:
examples, examples_persian = create_examples_prompt()
print(examples)
print("============================")
print()
print(examples_persian)

Example 1:
Review: این محصول از کیفیت مناسب و خوبی برخورداره از خریدش راضیم.
Sentiment: POSITIVE
Example 2:
Review: همه یکنواخت نمی پزند یکی له می شه و دیگری سفته پیشنهاد نمی کنم
Sentiment: NEGATIVE
Example 3:
Review: بسته بندی و طعم خوبی داره ، من دوست دارم واقعا
Sentiment: POSITIVE
Example 4:
Review: من اصلا نپسندیدم . همینطور مونده . اصلا با شربت خانگی قابل مقایسه نیست . من پیشنهاد نمی کنم .
Sentiment: NEGATIVE
Example 5:
Review: تعداد شیر های پرچرب کمتر از کم چرب بود ۴ تا کم چرب بود و ۲ تا پر چرب
Sentiment: NEUTRAL
Example 6:
Review: طعم خوبی نداره نسبت به حلوا شکری های بازار تنها حسنش تو شگفت انگیز فقط قیمتش از رقبا کمتره
Sentiment: OTHER


مثال 1:
نظر: این محصول از کیفیت مناسب و خوبی برخورداره از خریدش راضیم.
احساس: مثبت
مثال 2:
نظر: همه یکنواخت نمی پزند یکی له می شه و دیگری سفته پیشنهاد نمی کنم
احساس: منفی
مثال 3:
نظر: بسته بندی و طعم خوبی داره ، من دوست دارم واقعا
احساس: مثبت
مثال 4:
نظر: من اصلا نپسندیدم . همینطور مونده . اصلا با شربت خانگی قابل مقایسه نیست . من پیشنهاد نمی کن

### Prompt #01

In [71]:
for index, row in sample_prompts.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = run_model(context=context_01 + "Examples:\n" + examples, query=query_01.replace("SAMPLE_REVIEW", row['review']))
            print(f"Review: {row['review']}")
            print(f"Model Response: {' '.join(response)}")
            print(f"Actual Label: {label_dict[row['sentiment']]}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(5)

    if retries == max_retries:
        print(f"Failed to process this review after {max_retries} retries. Skipping to the next review.")


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Review: پودر قهوه دارای کیفیت مطلوب نبود وعطر و بوی در فضا پخش نمی شد
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: از این برند انتظار بیشتری داشتم خیلی خشک و بی مزه بود
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: تعداد شیر های پرچرب کمتر از کم چرب بود ۴ تا کم چرب بود و ۲ تا پر چرب
Model Response: negative
Actual Label: NEUTRAL
---------------------------------------------------------------------
Review: دفعه های قبل که سفارش دادم بو میداد ولی ایندفعه تازه بود و بوی بد هم نمیداد
Model Response: negative
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: به نظرم خیلی خوشمزه بود طعم شکلات با قهوه عالی میشه
Model Response: very positive
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: از نظر کیفیت خوبه ولی از نظر قیمت فقط تو پیشن

### Prompt #02

In [72]:
for index, row in sample_prompts.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = run_model(context=context_02 + "Examples:\n" + examples, query=query_02.replace("SAMPLE_REVIEW", row['review']))
            print(f"Review: {row['review']}")
            print(f"Model Response: {' '.join(response)}")
            print(f"Actual Label: {label_dict[row['sentiment']]}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(5)

    if retries == max_retries:
        print(f"Failed to process this review after {max_retries} retries. Skipping to the next review.")


Review: پودر قهوه دارای کیفیت مطلوب نبود وعطر و بوی در فضا پخش نمی شد
Model Response: positive
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: از این برند انتظار بیشتری داشتم خیلی خشک و بی مزه بود
Model Response: very negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: تعداد شیر های پرچرب کمتر از کم چرب بود ۴ تا کم چرب بود و ۲ تا پر چرب
Model Response: negative
Actual Label: NEUTRAL
---------------------------------------------------------------------
Review: دفعه های قبل که سفارش دادم بو میداد ولی ایندفعه تازه بود و بوی بد هم نمیداد
Model Response: negative
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: به نظرم خیلی خوشمزه بود طعم شکلات با قهوه عالی میشه
Model Response: very positive
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: از نظر کیفیت خوبه ولی از نظر قیمت فقط تو

### Prompt #03

In [73]:
for index, row in sample_prompts.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = run_model(context=context_03 + "Examples:\n" + examples, query=query_03.replace("SAMPLE_REVIEW", row['review']))
            print(f"Review: {row['review']}")
            print(f"Model Response: {' '.join(response)}")
            print(f"Actual Label: {label_dict[row['sentiment']]}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(5)

    if retries == max_retries:
        print(f"Failed to process this review after {max_retries} retries. Skipping to the next review.")


Review: پودر قهوه دارای کیفیت مطلوب نبود وعطر و بوی در فضا پخش نمی شد
Model Response: positive
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: از این برند انتظار بیشتری داشتم خیلی خشک و بی مزه بود
Model Response: very negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: تعداد شیر های پرچرب کمتر از کم چرب بود ۴ تا کم چرب بود و ۲ تا پر چرب
Model Response: negative
Actual Label: NEUTRAL
---------------------------------------------------------------------
Review: دفعه های قبل که سفارش دادم بو میداد ولی ایندفعه تازه بود و بوی بد هم نمیداد
Model Response: negative
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: به نظرم خیلی خوشمزه بود طعم شکلات با قهوه عالی میشه
Model Response: very positive
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: از نظر کیفیت خوبه ولی از نظر قیمت فقط تو

### Prompt #04

In [74]:
for index, row in sample_prompts.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = run_model(context=context_04 + "Examples:\n" + examples_persian, query=query_04.replace("SAMPLE_REVIEW", row['review']))
            print(f"Review: {row['review']}")
            print(f"Model Response: {' '.join(response)}")
            print(f"Actual Label: {label_dict[row['sentiment']]}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(5)

    if retries == max_retries:
        print(f"Failed to process this review after {max_retries} retries. Skipping to the next review.")


Review: پودر قهوه دارای کیفیت مطلوب نبود وعطر و بوی در فضا پخش نمی شد
Model Response: no sentiment expressed
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: از این برند انتظار بیشتری داشتم خیلی خشک و بی مزه بود
Model Response: no sentiment expressed
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: تعداد شیر های پرچرب کمتر از کم چرب بود ۴ تا کم چرب بود و ۲ تا پر چرب
Model Response: no sentiment expressed
Actual Label: NEUTRAL
---------------------------------------------------------------------
Review: دفعه های قبل که سفارش دادم بو میداد ولی ایندفعه تازه بود و بوی بد هم نمیداد
Model Response: no sentiment expressed
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: به نظرم خیلی خوشمزه بود طعم شکلات با قهوه عالی میشه
Model Response: negative
Actual Label: POSITIVE
---------------------------------------------------------------------
Re

### Prompt #05


In [75]:
for index, row in sample_prompts.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            response = run_model(context=row['review'], query=query_05)
            print(f"Review: {row['review']}")
            print(f"Model Response: {' '.join(response)}")
            print(f"Actual Label: {label_dict[row['sentiment']]}")
            print("---------------------------------------------------------------------")
            break
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retries += 1
            print(f"Retrying ({retries}/{max_retries})...")
            time.sleep(5)

    if retries == max_retries:
        print(f"Failed to process this review after {max_retries} retries. Skipping to the next review.")


Review: پودر قهوه دارای کیفیت مطلوب نبود وعطر و بوی در فضا پخش نمی شد
Model Response: negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: از این برند انتظار بیشتری داشتم خیلی خشک و بی مزه بود
Model Response: very negative
Actual Label: NEGATIVE
---------------------------------------------------------------------
Review: تعداد شیر های پرچرب کمتر از کم چرب بود ۴ تا کم چرب بود و ۲ تا پر چرب
Model Response: negative
Actual Label: NEUTRAL
---------------------------------------------------------------------
Review: دفعه های قبل که سفارش دادم بو میداد ولی ایندفعه تازه بود و بوی بد هم نمیداد
Model Response: negative
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: به نظرم خیلی خوشمزه بود طعم شکلات با قهوه عالی میشه
Model Response: very positive
Actual Label: POSITIVE
---------------------------------------------------------------------
Review: از نظر کیفیت خوبه ولی از نظر قیمت فقط تو

## Few-Shot Metric

### Prompt #01

In [76]:
few_01_actual = []
few_01_predicted = []
few_01_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = run_model(context=context_01 + "Examples:\n" + examples, query=query_01.replace("SAMPLE_REVIEW", row['review']))
                response = " ".join(response)
                few_01_actual.append(label_dict[row['sentiment']])
                few_01_responses.append(response)
                
                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        few_01_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    if "NO SENTIMENT EXPRESSED" in response:
                        few_01_predicted.append('NEUTRAL')
                    else:
                        few_01_predicted.append('OTHER')

                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()


|████████████████████████████████████████| 500/500 [100%] in 2:51.6 (2.91/s)    


In [77]:
few_01_results = calculate_metrics(few_01_actual, few_01_predicted)
few_01_results 

,OTHER,NEGATIVE,POSITIVE,NEUTRAL,All Classes
Accuracy,-,-,-,-,0.588
F1 Score,[0.0],[0.6678966789667897],[0.5920000000000001],[0.14814814814814814],0.551487
Precision,[0.0],[0.5370919881305638],[0.7816901408450704],[0.09523809523809523],0.585618
Recall,[0.0],[0.8829268292682927],[0.47639484978540775],[0.3333333333333333],0.588


In [78]:
np.save('few_01_actual', few_01_actual)

In [79]:
np.save('few_01_predicted', few_01_predicted)

In [80]:
np.save('few_01_responses', few_01_responses)

In [81]:
few_01_results.to_csv('few_01_results.csv',index=True)

### Prompt #02

In [82]:
few_02_actual = []
few_02_predicted = []
few_02_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = run_model(context=context_02 + "Examples:\n" + examples, query=query_02.replace("SAMPLE_REVIEW", row['review']))
                response = " ".join(response)
                few_02_actual.append(label_dict[row['sentiment']])
                few_02_responses.append(response)
                
                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        few_02_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    if "NO SENTIMENT EXPRESSED" in response:
                        few_02_predicted.append('NEUTRAL')
                    else:
                        few_02_predicted.append('OTHER')

                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()


on 0: /opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
        warnings.warn(


|████████████████████████████████████████| 500/500 [100%] in 2:58.3 (2.80/s)    


In [83]:
few_02_results = calculate_metrics(few_02_actual, few_02_predicted)
few_02_results

,OTHER,NEGATIVE,POSITIVE,NEUTRAL,All Classes
Accuracy,-,-,-,-,0.61
F1 Score,[0.0],[0.6854990583804143],[0.6318537859007832],[0.13333333333333333],0.577098
Precision,[0.0],[0.558282208588957],[0.8066666666666666],[0.08333333333333333],0.605802
Recall,[0.0],[0.8878048780487805],[0.51931330472103],[0.3333333333333333],0.61


In [84]:
np.save('few_02_actual', few_02_actual)

In [85]:
np.save('few_02_predicted', few_02_predicted)

In [86]:
np.save('few_02_responses', few_02_responses)

In [87]:
few_02_results.to_csv('few_02_results.csv',index=True)

### Prompt #03

In [88]:
few_03_actual = []
few_03_predicted = []
few_03_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = run_model(context=context_03 + "Examples:\n" + examples, query=query_03.replace("SAMPLE_REVIEW", row['review']))
                response = " ".join(response)
                few_03_actual.append(label_dict[row['sentiment']])
                few_03_responses.append(response)
                
                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        few_03_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    if "NO SENTIMENT EXPRESSED" in response:
                        few_03_predicted.append('NEUTRAL')
                    else:
                        few_03_predicted.append('OTHER')

                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()


on 0: /opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
        warnings.warn(


|████████████████████████████████████████| 500/500 [100%] in 2:58.3 (2.80/s)    


In [89]:
few_03_results = calculate_metrics(few_03_actual, few_03_predicted)
few_03_results

,OTHER,NEGATIVE,POSITIVE,NEUTRAL,All Classes
Accuracy,-,-,-,-,0.556
F1 Score,[0.0],[0.6567164179104478],[0.5479452054794521],[0.09302325581395349],0.525712
Precision,[0.0],[0.5317220543806647],[0.7575757575757576],[0.05405405405405406],0.571685
Recall,[0.0],[0.8585365853658536],[0.4291845493562232],[0.3333333333333333],0.556


In [90]:
np.save('few_03_actual', few_03_actual)

In [91]:
np.save('few_03_predicted', few_03_predicted)

In [92]:
np.save('few_03_responses', few_03_responses)

In [93]:
few_03_results.to_csv('few_03_results.csv',index=True)

### Prompt #04

In [94]:
few_04_actual = []
few_04_predicted = []
few_04_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = run_model(context=context_04 + "Examples:\n" + examples_persian, query=query_04.replace("SAMPLE_REVIEW", row['review']))
                response = " ".join(response)
                few_04_actual.append(label_dict[row['sentiment']])
                few_04_responses.append(response)
                
                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        few_04_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    if "NO SENTIMENT EXPRESSED" in response:
                        few_04_predicted.append('NEUTRAL')
                    else:
                        few_04_predicted.append('OTHER')

                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()


on 0: /opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
        warnings.warn(


|████████████████████████████████████████| 500/500 [100%] in 3:15.5 (2.56/s)    


In [95]:
few_04_results = calculate_metrics(few_04_actual, few_04_predicted)
few_04_results

,OTHER,NEGATIVE,POSITIVE,NEUTRAL,All Classes
Accuracy,-,-,-,-,0.264
F1 Score,[0.0],[0.38968481375358166],[0.3856209150326797],[0.03460207612456747],0.339885
Precision,[0.0],[0.4722222222222222],[0.8082191780821918],[0.0176678445229682],0.570453
Recall,[0.0],[0.33170731707317075],[0.2532188841201717],[0.8333333333333334],0.264


In [96]:
np.save('few_04_actual', few_04_actual)

In [97]:
np.save('few_04_predicted', few_04_predicted)

In [98]:
np.save('few_04_responses', few_04_responses)

In [99]:
few_04_results.to_csv('few_04_results.csv',index=True)

### Prompt #05

In [100]:
few_05_actual = []
few_05_predicted = []
few_05_responses = []

with alive_bar(test_count, force_tty=True) as bar:
    for index, row in test_df.iterrows():
        retries = 0
        while retries < max_retries:
            try:
                response = run_model(context=row['review'], query= "Examples:\n" + examples + query_05)
                response = " ".join(response)
                few_05_actual.append(label_dict[row['sentiment']])
                few_05_responses.append(response)
                
                flag = False
                response = response.upper()
                for label in acceptable_classes:
                    if label in response:
                        few_05_predicted.append(label)
                        flag = True
                        break

                if not flag:
                    if "NO SENTIMENT EXPRESSED" in response:
                        few_05_predicted.append('NEUTRAL')
                    else:
                        few_05_predicted.append('OTHER')

                break

            except Exception as e:
                print(f"An error occurred: {str(e)}")
                retries += 1
                print(f"Retrying ({retries}/{max_retries})...")

        if retries == max_retries:
            print(f"Failed to process review after {max_retries} retries. Skipping to the next entry.")

        bar()


on 0: /opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
        warnings.warn(


|████████████████████████████████████████| 500/500 [100%] in 3:03.2 (2.73/s)    


In [101]:
few_05_results = calculate_metrics(few_05_actual, few_05_predicted)
few_05_results

,OTHER,NEGATIVE,POSITIVE,NEUTRAL,All Classes
Accuracy,-,-,-,-,0.674
F1 Score,[0.15873015873015872],[0.7153846153846155],[0.7104622871046229],[0.0],0.642161
Precision,[0.7142857142857143],[0.5904761904761905],[0.8202247191011236],[0.0],0.70432
Recall,[0.08928571428571429],[0.9073170731707317],[0.6266094420600858],[0.0],0.674


In [102]:
np.save('few_05_actual', few_05_actual)

In [103]:
np.save('few_05_predicted', few_05_predicted)

In [104]:
np.save('few_05_responses', few_05_responses)

In [105]:
few_05_results.to_csv('few_05_results.csv',index=True)

## Boula

In [106]:
# run_model(
#     "یک فیلم ضعیف بی محتوا بدون فیلمنامه . شوخی های سخیف .",
#     "نظر شما در مورد داستان، فیلمنامه، دیالوگ ها و موضوع فیلم  لونه زنبور چیست؟"
# )

# run_model(
#     "فیلم تا وسط فیلم یعنی دقیقا تا جایی که معلوم میشه بچه های املشی دنبال رضان خیلی خوب و جذاب پیش میره ولی دقیقا از همونجاش سکته میزنه و خلاص...",
#     "نظر شما به صورت کلی در مورد فیلم  ژن خوک چیست؟"
# )
# run_model(
#     "اصلا به هیچ عنوان علاقه نداشتم اجرای می سی سی پی نشسته میمیرد روی پرده سینما ببینم  دیالوگ های تکراری   هلیکوپتر  ماشین  آلندلون  لئون  پاپیون  آخه چرااااااااااااااا   همون حسی که توی تالار وحدت بعد از نیم ساعت به سرم اومد امشب توی سالن سینما تجربه کردم ،حس گریز از سالن....... (ノಠ益ಠ)ノ ",
#     " نظر شما در مورد صداگذاری و جلوه های صوتی فیلم  مسخره‌باز چیست؟"
# )

# run_model(
#     " گول نخورید این رنگارنگ مینو نیست برای شرکت گرجیه و متاسفانه این محصولش اصلا مزه رنگارنگی که انتظار دارید رو نمیده ",
#     " نظر شما در مورد عطر، بو، و طعم این بیسکویت و ویفر چیست؟"
# )

# run_model(
#     "در مقایسه با سایر برندهای موجود در بازار با توجه به حراجی که داشت ارزانتر ب",
#     " شما در مورد قیمت و ارزش خرید این حبوبات و سویا چیست؟"
# )

# run_model(
#     "من پسرم عاشق ایناس ولی دیگه به خاطر حفظ محیط زیست فقط زمانهایی که مجبور باشم شیر دونه ای میخرم و سعی میکنم دیگه کمتر شیر با بسته بندی تتراپک استفاده کنم ",
#     "نظر شما به صورت کلی در مورد این شیر چیست؟"
# )
